<a href="https://colab.research.google.com/github/acouprie/vehicule-tour/blob/master/Routing_Optimization_Idai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Routing optimization in a flooding context

## Description of the project
In the context of the e-flooding project, the SEPIA team at IRIT is looking for a motivated student team working on the integration of a territory map and an algorithm of planning of civil security vehicles movement. This part of the project is called “Vehicle Tours” and should simulate emergency services interventions during a flood, which are deployed on the territory to save people, animals and goods.
## Content
The E-flooding project is a simulation of a flooding emergency services with constraints of resources such as vehicules (with a certain amount of places, an ability to drive only on roads, water or on airs) and the emergency stations, where they start from. Those saving point might change during the time if needed.
The vehicules needs to go at a precise location to save people in distress. Those saving zones are called challenge, in other words buildings where the emergency services need to have an intervention (hospitals, schools…).
The vehicles should opere a optimized circuit, such that they can pass through as much as challenge possible according to the following constraints: blocked roads, vehicle type and capacity.
## Goals
The goal of the product to be delivered is to implement an interaction between VROOM and QGIS, and adapting VROOM to run a custom algorithm and apply it to a predefined graph.
## Customer constraints
The project should be “Plug and Play”, that means can works with any map and algorithm without new configuration. The project should also be portable, that means can be executed easily on any computer, to achieve that, the solution picked is Docker in order to deliver the project in containers.
There is no limit in terms of financial spend into the wages and the numbers of hours worked.
## Technical specifications
The project will be developed using the programming language Python 3 which will interface VROOM tool with Openstreetmap and QGIS, a geographical information system.
The program should be able to take as an input a graph representing the different challenges.

We'll solve this complexe problem in this Python 3 Jupyter Notebook with [OpenRouteService](https://openrouteservice.org) new **route optimization service** which implement VROOM.

This Jupiter Notebook get its inspiration from the two awesome example projects of ORS [Disaster optimization](https://openrouteservice.org/disaster-optimization/) and [Avoided flooded areas](https://openrouteservice.org/example-avoid-flooded-areas-with-ors/).

## Requirements

You will need two .csv input files, one for the vehicle fleet and one for the concerns. Those files can be found on the resources folder of the [GitHub](https://github.com/acouprie/vehicule-tour) project.

## The logistics setup

In total 20 sites were identified in need of public emergency services intervention, while 3 vehicles were scheduled for the operation.

The **vehicles** were all located in the Stade de France at Paris, France and had the same following constraints:

- operation time windows from 8:00 to 20:00
- loading capacity:
  "car" : 5,
  "bus" : 50,
  "boat" : 10,
  "helicopter" : 3

The **concerns** are located in the Île de France region. Their needs range from 1 to 5 people to save and have a constraint for time window of the intervention (from 8:00 min to 20:00 max).

In [0]:
!pip install openrouteservice
!pip install pyproj

import folium
from folium.plugins import BeautifyIcon
import shapely
import pandas as pd
import openrouteservice as ors
from IPython.core.interactiveshell import InteractiveShell

from shapely import geometry
from shapely.geometry import shape, Polygon, mapping, MultiPolygon, LineString, Point
from shapely.ops import cascaded_union, transform

import os
import pyproj
import time
import random

In [0]:
# Examples files can be downloaded there: https://github.com/acouprie/vehicule-tour/tree/master/resources
needed_files = ['vehicle_list.csv', 'concern_list.csv', 'flood_list.csv']

if set(needed_files).issubset(set(os.listdir())):
    replace = 'N' #@param {type:"string"}
    print("Replace the files ? (Y/N) ")
    if(replace == 'y' or replace == 'Y'):
        !rm needed_files[0]
        !rm needed_files[1]
        !rm needed_files[2]
        from google.colab import files
        uploaded_csv = files.upload()
else:
    from google.colab import files
    uploaded_csv = files.upload()

In [0]:
# For production purpose, please get your own ORS API key
clnt = ors.Client(key='5b3ce3597851110001cf624855844106946f43c59ce6e46f048c554d')

## The routing problem setup

Now that we have described the setup sufficiently, we can start to set up our actual Vehicle Routing Problem. For this example we're using the FOSS library of [Vroom](https://github.com/VROOM-Project/vroom), which has [recently seen](http://k1z.blog.uni-heidelberg.de/2019/01/24/solve-routing-optimization-with-vroom-ors/) support for OpenRouteService and is available through their APIs.

To properly describe the problem in algorithmic terms, we have to provide the following information:

- **vehicles start/end address**: vehicle depot in Stade de France at Paris, France
- **vehicle capacity**:  "car" : 5, "bus" : 50, "boat" : 10, "helicopter" : 3
- **vehicle operational times**: 08:00 - 20:00
- **service location**: pick-up location
- **service time windows**: individual pick-up location's time window
- **service amount**: individual pick-up location's needs

We defined all these parameters in the data sheets `concern_list.csv` and `vehicle_list.csv`. Now we have to only wrap this informatoin into our code and send a request to OpenRouteService optimization service at [`https://api.openrouteservice.org/optimization`](https://openrouteservice.org/dev/#/api-docs/optimization/post).

In [0]:
# The vehicles are all located at the safe point Stade de France, Paris, France
depot = [48.9244725,2.3601325]

# Define vehicle class
class Vehicle :
    profile = "Vehicle"
    cap = 50
    start = list(reversed(depot))
    end = list(reversed(depot))
    time_window = [1553241600, 1553284800]  # Fri 8-20:00, expressed in POSIX timestamp

switcherC = {
    "car" : 5,
    "bus" : 50,
    "boat" : 10,
    "helicopter" : 3
}

# Define the map centered on Paris
m = folium.Map(location=depot, tiles='cartodbpositron', zoom_start=11)

# Function to create buffer around tweet point geometries and transform it to the needed coordinate system (WGS84)
def CreateBufferPolygon(point_in, resolution=2, radius=20):    
    sr_wgs = pyproj.Proj(init='epsg:4326') # WGS84
    sr_utm = pyproj.Proj(init='epsg:32632') # UTM32N
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in) # Unpack list to arguments
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution) # 20 m buffer
    
    # Iterate over all points in buffer and build polygon
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point)) # Transform back to WGS84 
    return poly_wgs

# Function to request directions with avoided_polygon feature
def CreateRoute(avoided_point_list, coordinates):
    route_request = {'coordinates': coordinates, 
                    'format_out': 'geojson',
                    'profile': 'driving-car',
                    'preference': 'shortest',
                    'instructions': False,
                    'options': {'avoid_polygons': geometry.mapping(MultiPolygon(avoided_point_list))}} 
    route_directions = clnt.directions(**route_request)
    
    return route_directions

# To style data
def style_function(color):
    return lambda feature: dict(color=color)

In [0]:
# Get input vehicles from the file vehicle_list
vehicles_data = pd.read_csv('./vehicle_list.csv')

# Define the vehicles
vehicles = list()
helicopters = list()
# Sort the helicopters from others vehicles as they are not optimized
for vehicle in vehicles_data.itertuples():
    if(vehicle.Type == "helicopter"):
        helicopters.append(
            {
                "id": vehicle.ID,
                "start": Vehicle.start,
                "end": Vehicle.end,
                "go_save": vehicle.Go_Save,
                "capacity": switcherC.get(vehicle.Type,50),
                "time_window": Vehicle.time_window
            }
        )
    else:
        vehicles.append (
            ors.optimization.Vehicle(
                id = vehicle.ID, 
                start = Vehicle.start,
                end = Vehicle.end,
                capacity = [switcherC.get(vehicle.Type,50)],
                time_window = Vehicle.time_window
            )
        )

In [0]:
# Next define the concern places
concerns_data = pd.read_csv(
    './concern_list.csv',
    index_col="ID",
    parse_dates=["Save_From", "Save_Before"]
)

concerns = list()
for concern in concerns_data.itertuples():
    concerns.append(
        ors.optimization.Job(
            id=concern.Index,
            location=[concern.Lon, concern.Lat],
            service=1200,  # Assume 20 minutes at each site
            amount=[concern.Needed_Amount],
            time_windows=[[
                int(concern.Save_From.timestamp()),  # VROOM expects UNIX timestamp
                int(concern.Save_Before.timestamp())
            ]]
        )
    )

# Plot the concerns and the safe point on the map with more info in the ToolTip
for job in concerns:
    tooltip = folium.map.Tooltip("<h4><b>ID {}</b></p><p>Persons to save : <b>{}</b></p>".format(
        job.id, job.amount
    ))
    
    folium.Marker(
        location=[job.location[1], job.location[0]],
        tooltip=tooltip,
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(job.id),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(m)    

folium.Marker(
    location=depot,
    icon=folium.Icon(color="green", icon="heart empty", prefix='fa')
).add_to(m)

In [0]:
# Define the flooded areas and plot them on the map
floods_data = pd.read_csv('./flood_list.csv')
floods = list()
for flood in floods_data.itertuples():
    radius = flood.Radius
    if radius is None:
        radius = 1000
    floods.append(
        {
            "id": flood.ID,
            "longitude": flood.Lon,
            "latitude": flood.Lat,
            "radius": radius
        }
    )
    
flood_areas = list()
avoided_point_list = list()
for area in floods:
    flood_area = CreateBufferPolygon(
                    [area["longitude"], area["latitude"]],
                    resolution = 5,
                    radius = 1000
                   )
    flood_areas.append(flood_area)
    poly = Polygon(flood_area)
    avoided_point_list.append(poly) # This area will be avoided
big_poly = cascaded_union(avoided_point_list)
union_poly = mapping(big_poly)
folium.features.GeoJson(data=union_poly,
                        name='Flood affected areas',
                        style_function=style_function('#1d4591')
                        ).add_to(m)

## Helicopters work

In [0]:
point_in_water = list()
for concern in concerns:
    c_pt = Point(concern.location)
    if (c_pt.within(big_poly)):
        point_in_water.append(concern)
        concerns.remove(concern)

# Manage helicos out of the optimization
while point_in_water:
    for helico in helicopters:
        if not point_in_water:
            break
        concern = point_in_water[0]
        concern_location = [concern.location[1], concern.location[0]]
        helico_trip = [depot, concern_location]
        concern.amount[0] -= helico["capacity"]
        # If the concern is empty, we remove it from the optimization
        if(concern.amount[0] <= 0):
            del point_in_water[0]
        folium.PolyLine(
            helico_trip,
            tooltip="<h4>Helicopter " + str(helico["id"]) + "</h4>",
            color="#%06x" % random.randint(0, 0xFFFFFF),
            weight=2.5,
            opacity=1
        ).add_to(m)

## Driving vehicle work

In [0]:
paths = list()
vehicle_index = list()

while concerns :
  # Make the request
  result = clnt.optimization(
      jobs=concerns,
      vehicles=vehicles,
      geometry=True
  )

  # Create a list to display the location of all concerns taken by each vehicule
  concerns_id = list()
  for route in result['routes']:
      vehicle_job = list()
      vehicle_index.append(route.get("vehicle"))
      for step in route["steps"]:
          if(step.get("job") != None):
              concerns_id.append(step.get("job"))
          vehicle_job.append(
          [
            step.get("job", "Safe Point"),  # Station ID
            step["location"]  # Location of the job
          ]
          )
      paths.append(vehicle_job)
  for c_id in concerns_id :
    for concern in concerns :
      if (c_id == concern.id) :
        concerns.remove(concern)

### Avoid flood areas
With a first set of generated routes, we use the pairs vehicle-concern to modify the routes to avoid the flood zones.

In [0]:
# Color generator to print the routes
colors = list()
for _ in paths:
    colors.append("#%06x" % random.randint(0, 0xFFFFFF))

# Get coordinates
all_coordinates = list()
for vehicle_path in paths:
    vehicle_coordinates = list()
    for job in vehicle_path:
        vehicle_coordinates.append(job[1]) #job[0] = ID and job[1] = job location
    all_coordinates.append(vehicle_coordinates)

In [0]:
for color, vehicle, v_id in zip(colors, all_coordinates,vehicle_index):
  for element in range (0, (len(vehicle)-1)):
      try:
        coordinates = [vehicle[element],vehicle[element+1]] # coordinates has, for the vehicle 0, his start point (0) and the first point to arrive (1)
        route_directions = CreateRoute(avoided_point_list, coordinates)
        folium.features.GeoJson(data=route_directions, 
                                name='Alternative Route',
                                style_function=style_function(color),
                                overlay=True,
                                tooltip=folium.Tooltip("<h4>Vehicle " + str(v_id) + "</h4>")
                                ).add_to(m)
        # ORS has a rate limit of 40 requests per seconds
        time.sleep(0.5)
      except Exception:
        print('Exception : could not create a new route')

# Display the map
m

## Data view

### Path 0 details
Below you can observe the concerns reached by the vehicle 0. To change the path you only need to pass in `paths[0]` the path number that you want.

In [0]:
path_v0 = pd.DataFrame(paths[0], columns=["Station ID", "Location"])
path_v0

,Station ID,Location
0,Safe Point,"[2.3601325, 48.9244725]"
1,8,"[2.28528835878907, 48.8202623111443]"
2,9,"[2.40836956606446, 48.7996873840938]"
3,Safe Point,"[2.3601325, 48.9244725]"


Alfaro Romero Sandra - Amiard Landry - Couprie Antoine <br />
University Paul Sabatier, Toulouse - France